<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-films/blob/analiza_b%C5%82%C4%99d%C3%B3w/colabs/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Wczytywanie danych i instalacje początkowe

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark2.4.5
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
# install findspark
!pip install -q findspark
# clone github repo
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-films
# Przeniesienie plików z BigData2022-films do katalogu nadrzędnego
!mv BigData2022-films/* .
!mv BigData2022-films/.* .
!rmdir BigData2022-films

Cloning into 'BigData2022-films'...
remote: Enumerating objects: 1321, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 1321 (delta 12), reused 12 (delta 4), pack-reused 1295
Receiving objects: 100% (1321/1321), 2.75 MiB | 14.38 MiB/s, done.
Resolving deltas: 100% (829/829), done.
mv: cannot move 'BigData2022-films/.' to './.': Device or resource busy
mv: cannot move 'BigData2022-films/..' to './..': Device or resource busy


In [3]:
import os

# setup environment variables for our Spark Session to work
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.2.1-bin-hadoop3.2'

from lib.pyspark_startup import init, load

In [4]:
!git checkout analiza_błędów

Branch 'analiza_błędów' set up to track remote branch 'analiza_błędów' from 'origin'.
Switched to a new branch 'analiza_błędów'


In [5]:
spark = init()

In [6]:
path = "/content/drive/.shortcut-targets-by-id/1VcOir9FMG8LzEsUE-Q8YA79c_sV0tJwp/bigdata2022/"

In [7]:
test = spark.read.parquet(path + "test_df.parquet")
train = spark.read.parquet(path + "train_df.parquet")
valid = spark.read.parquet(path + "valid_df.parquet")

In [8]:
df_temp = test.union(train)
df_clean = df_temp.union(valid)

In [9]:
import pyspark.sql.functions as f
from pyspark.sql.types import FloatType 
import random
from pyspark.sql import DataFrame

## Funkcje

In [11]:
#TODO Przeniesc ponizsze funkcje do liba, najlepiej oddzielny plik np. pyspark_demo.py

def testujemy(df,a_param:float):

    add_udf = f.udf(lambda x, y: round(a_param* x + (1- a_param) * y,4), FloatType())
    df = df.withColumn("prediction", add_udf(df["cos_similarity"], df["IOU"]))

    return df

In [10]:
from pyspark.sql.functions import desc
from lib.pyspark_matrix_similarity import cos_sim_and_iou_for_row
from lib.ranking_function import ranking_list

def najbardziej_podobne (id, ile, df1):
  #wyrzuca najlepsze wartości prediction
    df = cos_sim_and_iou_for_row(df1,id)
    df_prediction= testujemy(df, 0.5)
    df_prediction = df_prediction.sort(desc("prediction"))
    df_prediction.show(ile, truncate=False) 

## Jak wyglądają nasze dane?

Podejrzyjmy zbiór testowy:

In [50]:
test.show(10, truncate=False)

+---------+----------------------------------------------------------+---------------------+-------------------------+-----------------------+--------------+--------------------------+---------------------------------------------------------------------------------------------------------+------------------------------------------+
|id       |tytul                                                     |rok_wydania_produkcji|rok_zakonczenia_produkcji|dlugosc_produkcji_w_min|liczba_sezonow|liczba_wszystkich_odcinkow|ludzie_filmu                                                                                             |features                                  |
+---------+----------------------------------------------------------+---------------------+-------------------------+-----------------------+--------------+--------------------------+---------------------------------------------------------------------------------------------------------+------------------------------------------

Ciekawie wyglądają ostatnie dwie kolumny - ludzie_filmu zawiera 10-elementową listę wypełnioną najbardziej znaczącymi osobami przy produkcji filmu (np. reżyser, aktorzy pierwszoplanowi). 
Kolumna features zawiera Sparse vector cech, które używamy w naszym similarity. Są to sklejone kolejno wektory:

*   czy_dla_dorosłych (długość 1)
*   rodzaj_produkcji_ohe (długość 7)
*   epoka_rok_wydania_produkcji_ohe (długość 9)
*   gatunek_vec (długość 28)

Na przykładzie filmu "Blacksmith scene" wektor features ma długość 45, na jego 
1, 17 oraz 20 pozycji są 1, co oznacza rodzaj produkcji = short, epokę 1 oraz gatunek = komedia.

## Testowanie

Zobaczmy jak nasz kod działa w praktyce. 

### Toy Story

In [15]:
najbardziej_podobne("tt0114709",15,df_clean)

+---------+-----------------------------------------------------+--------------+----------+----------+
|id       |tytul                                                |cos_similarity|IOU       |prediction|
+---------+-----------------------------------------------------+--------------+----------+----------+
|tt0114709|Toy Story                                            |1.0           |1.0       |1.0       |
|tt0120363|Toy Story 2                                          |1.0           |0.33333334|0.6667    |
|tt0120623|A Bug's Life                                         |1.0           |0.1764706 |0.5882    |
|tt0317219|Cars                                                 |1.0           |0.11111111|0.5556    |
|tt0338348|The Polar Express                                    |1.0           |0.05263158|0.5263    |
|tt0429589|The Ant Bully                                        |1.0           |0.05263158|0.5263    |
|tt1049413|Up                                                   |1.0     

In [75]:
ranking_list(df_clean,"tt0114709").sort_values(by='score', ascending=False).head(15)

,id,score
161790,tt0114709,18.0
163342,tt0120363,13.0
623219,tt0120623,11.0
681047,tt0435761,10.0
839037,tt1979376,10.0
106929,tt2446040,9.0
221827,tt0429589,9.0
1046692,tt0198781,9.0
659889,tt0317219,9.0
666156,tt0356634,9.0


Zgodnie z intuicją (a także funkcją rankującą), kolejne części powinny być bardziej podobne do Toy Story - jest to kontynuacja tej samej serii, występują ci sami bohaterowie. Można jednak wytłumaczyć tak odległe miejsce w rankingu faktem, że części te powstały na przestrzeni 23 lat, a bohaterowie tej serii dorastają wraz z widzem, a ich problemy i sposób postrzegania świata zmienia się w kolejnych częściach.

### DC

Ciekawie wyglądają również filmy z wydawnictwa DC związane z postacią Batmana.  Po sprawdzeniu najbardziej podobnych filmów do "Mrocznego rycerza" otrzymujemy "Batman: początek" oraz "Mroczny Rycerz powstaje". Co ciekawe nie znajduje znaczącego podobieństwa do filmu "Batman" z 2022 roku (analogicznie w druga stronę).

In [52]:
najbardziej_podobne("tt0468569",10,df_clean)

+---------+---------------------+--------------+----------+----------+
|id       |tytul                |cos_similarity|IOU       |prediction|
+---------+---------------------+--------------+----------+----------+
|tt0468569|The Dark Knight      |1.0           |1.0       |1.0       |
|tt0372784|Batman Begins        |1.0           |0.53846157|0.7692    |
|tt1345836|The Dark Knight Rises|0.6708204     |0.42857143|0.5497    |
|tt0963178|The International    |1.0           |0.05263158|0.5263    |
|tt0433387|Harsh Times          |1.0           |0.05263158|0.5263    |
|tt0119099|Fallen               |1.0           |0.05263158|0.5263    |
|tt1289406|Harry Brown          |1.0           |0.05263158|0.5263    |
|tt0381849|3:10 to Yuma         |1.0           |0.05263158|0.5263    |
|tt0367878|Os Imortais          |1.0           |0.0       |0.5       |
|tt0214555|Bullets Over Summer  |1.0           |0.0       |0.5       |
+---------+---------------------+--------------+----------+----------+
only s

In [53]:
najbardziej_podobne("tt1877830",10,df_clean)

+----------+--------------------------------+--------------+----------+----------+
|id        |tytul                           |cos_similarity|IOU       |prediction|
+----------+--------------------------------+--------------+----------+----------+
|tt1877830 |The Batman                      |1.0           |1.0       |1.0       |
|tt14857194|Superman: Agent of Batman       |1.0           |0.11111111|0.5556    |
|tt3156558 |Caped Crusader: The Dark Hours  |1.0           |0.05882353|0.5294    |
|tt1235522 |Broken City                     |1.0           |0.05263158|0.5263    |
|tt5057140 |Hold the Dark                   |1.0           |0.05263158|0.5263    |
|tt2602174 |Batman Revealed                 |1.0           |0.05263158|0.5263    |
|tt3647498 |Blood Father                    |1.0           |0.05263158|0.5263    |
|tt2101341 |Dead Man Down                   |1.0           |0.05263158|0.5263    |
|tt2345737 |The Rover                       |1.0           |0.05263158|0.5263    |
|tt1

### Władca pierścieni

Nasze prediction dla Władcy Pierścieni: Powrót Króla wskazuje jako bardziej podobne pozostałe 2 części trylogii, niż wersję reżyserską tego samego filmu (czyli rozszerzoną, z dodatkowymi scenami). Po weryfikacji danych na IMDB może wynikać to z faktu, że dla wersji reżyserskiej mamy mniej informacji, np. w obsadzie wersji reżyserskiej występuje tylko Elijah Wood, gdzie dla podstawowej wersji filmu jest to kilkadziesiąt osób.

In [55]:
najbardziej_podobne("tt0167260",10,df_clean)

+----------+-----------------------------------------------------------------------------------+--------------+----------+----------+
|id        |tytul                                                                              |cos_similarity|IOU       |prediction|
+----------+-----------------------------------------------------------------------------------+--------------+----------+----------+
|tt0167260 |The Lord of the Rings: The Return of the King                                      |1.0           |1.0       |1.0       |
|tt0167261 |The Lord of the Rings: The Two Towers                                              |1.0           |0.8181818 |0.9091    |
|tt0120737 |The Lord of the Rings: The Fellowship of the Ring                                  |1.0           |0.6666667 |0.8333    |
|tt21811588|The Lord of the Rings: The Fellowship of the Ring - Special Extended Edition Scenes|0.8           |0.54545456|0.6727    |
|tt21811606|The Lord of the Rings: The Return of the King - Sp

### Ojciec chrzestny

Dla "Ojca chrzestnego" nasz algorytm wskazuje jako najbardziej podobne filmy gangsterskie.

In [56]:
najbardziej_podobne("tt0068646",10,df_clean)

+---------+---------------------------------+--------------+----------+----------+
|id       |tytul                            |cos_similarity|IOU       |prediction|
+---------+---------------------------------+--------------+----------+----------+
|tt0068646|The Godfather                    |1.0           |1.0       |1.0       |
|tt0071562|The Godfather: Part II           |1.0           |0.42857143|0.7143    |
|tt0099674|The Godfather: Part III          |0.75          |0.33333334|0.5417    |
|tt0086442|A Time to Die                    |1.0           |0.05263158|0.5263    |
|tt0086250|Scarface                         |1.0           |0.05263158|0.5263    |
|tt0086066|The Outsiders                    |1.0           |0.05263158|0.5263    |
|tt0071532|The Gambler                      |1.0           |0.05263158|0.5263    |
|tt0087089|The Cotton Club                  |0.8944272     |0.11111111|0.5028    |
|tt0073845|L'uomo che sfidò l'organizzazione|1.0           |0.0       |0.5       |
|tt0

### Harry Potter i Insygnia Śmierci: część 2

Dla filmu Harry Potter i Insygnia Śmierci: część 2 similarity wskazuje, że najbardziej podobne są części, które chronologicznie znajdują się najbliżej. Jest to zasadne, ponieważ film "dorastał z widzem" i można zauważyć, że każda kolejna część była coraz bardziej mroczna. W zestawieniu pojawiła się również seria "Fantastyczne Zwierzęta" - również jest to uzasadnione, ponieważ jest ona osadzona w tym samym uniwersum, a klimat jest podobny to późniejszych części serii.

In [59]:
najbardziej_podobne("tt1201607",15,df_clean)

+----------+--------------------------------------------+--------------+----------+----------+
|id        |tytul                                       |cos_similarity|IOU       |prediction|
+----------+--------------------------------------------+--------------+----------+----------+
|tt1201607 |Harry Potter and the Deathly Hallows: Part 2|1.0           |1.0       |1.0       |
|tt0926084 |Harry Potter and the Deathly Hallows: Part 1|1.0           |0.8181818 |0.9091    |
|tt0417741 |Harry Potter and the Half-Blood Prince      |0.6           |0.8181818 |0.7091    |
|tt3183660 |Fantastic Beasts and Where to Find Them     |1.0           |0.25      |0.625     |
|tt4123430 |Fantastic Beasts: The Crimes of Grindelwald |1.0           |0.25      |0.625     |
|tt4123432 |Fantastic Beasts: The Secrets of Dumbledore |1.0           |0.25      |0.625     |
|tt0330373 |Harry Potter and the Goblet of Fire         |0.8           |0.42857143|0.6143    |
|tt0304141 |Harry Potter and the Prisoner of Azkab

### Dobry, zły, brzydki

In [76]:
najbardziej_podobne("tt0060196",15,df_clean)

+---------+------------------------------+--------------+----------+----------+
|id       |tytul                         |cos_similarity|IOU       |prediction|
+---------+------------------------------+--------------+----------+----------+
|tt0060196|The Good, the Bad and the Ugly|1.0           |1.0       |1.0       |
|tt0059578|For a Few Dollars More        |0.8660254     |0.25      |0.558     |
|tt0061152|Legacy of the Incas           |1.0           |0.05263158|0.5263    |
|tt0064208|Death Rides a Horse           |0.8660254     |0.1764706 |0.5212    |
|tt0057639|The Sign of the Coyote        |1.0           |0.0       |0.5       |
|tt0057687|Winnetou                      |1.0           |0.0       |0.5       |
|tt0063847|Zorro the Fox                 |1.0           |0.0       |0.5       |
|tt0062709|Bang Bang Kid                 |1.0           |0.0       |0.5       |
|tt0057484|Duel at the Rio Grande        |1.0           |0.0       |0.5       |
|tt0056652|Zorro the Avenger            

### Pewnego razu na Dzikim Zachodzie

Dla tytułu "Pewnego razu na Dzikim Zachodzie" wskazane podobne filmy wszystkie należą do gatunku western.

In [78]:
najbardziej_podobne("tt0064116",10,df_clean)

+---------+------------------------------+--------------+-----------+----------+
|id       |tytul                         |cos_similarity|IOU        |prediction|
+---------+------------------------------+--------------+-----------+----------+
|tt0064116|Once Upon a Time in the West  |1.0           |1.0        |1.0       |
|tt0059578|For a Few Dollars More        |1.0           |0.1764706  |0.5882    |
|tt0060165|A Big Hand for the Little Lady|1.0           |0.11111111 |0.5556    |
|tt0061636|Face to Face                  |1.0           |0.11111111 |0.5556    |
|tt0061787|Hour of the Gun               |1.0           |0.055555556|0.5278    |
|tt0063501|The Big Gundown               |1.0           |0.05263158 |0.5263    |
|tt0052194|Showdown at Boot Hill         |1.0           |0.05263158 |0.5263    |
|tt0056085|How the West Was Won          |1.0           |0.05263158 |0.5263    |
|tt0051087|The Tin Star                  |1.0           |0.05263158 |0.5263    |
|tt0058475|Bullets Don't Arg

### Marvel

In [84]:
najbardziej_podobne("tt4154756",10,df_clean)

+---------+-----------------------------------+--------------+----------+----------+
|id       |tytul                              |cos_similarity|IOU       |prediction|
+---------+-----------------------------------+--------------+----------+----------+
|tt4154756|Avengers: Infinity War             |1.0           |1.0       |1.0       |
|tt4154796|Avengers: Endgame                  |0.8           |1.0       |0.9       |
|tt3498820|Captain America: Civil War         |1.0           |0.53846157|0.7692    |
|tt2395427|Avengers: Age of Ultron            |1.0           |0.42857143|0.7143    |
|tt1843866|Captain America: The Winter Soldier|1.0           |0.42857143|0.7143    |
|tt0458339|Captain America: The First Avenger |1.0           |0.25      |0.625     |
|tt1300854|Iron Man 3                         |1.0           |0.1764706 |0.5882    |
|tt1228705|Iron Man 2                         |1.0           |0.1764706 |0.5882    |
|tt1502712|Fantastic Four                     |1.0           |0.1

In [82]:
najbardziej_podobne("tt1431045",10,df_clean)

+----------+--------------------------------------------------------------+--------------+----------+----------+
|id        |tytul                                                         |cos_similarity|IOU       |prediction|
+----------+--------------------------------------------------------------+--------------+----------+----------+
|tt1431045 |Deadpool                                                      |1.0           |1.0       |1.0       |
|tt5463162 |Deadpool 2                                                    |1.0           |0.33333334|0.6667    |
|tt5884052 |Pokémon: Detective Pikachu                                    |1.0           |0.05263158|0.5263    |
|tt6264654 |Free Guy                                                      |1.0           |0.05263158|0.5263    |
|tt0790736 |R.I.P.D.                                                      |1.0           |0.05263158|0.5263    |
|tt2463208 |The Adam Project                                              |1.0           |0.0526

In [83]:
najbardziej_podobne("tt0325980",10,df_clean)

+---------+------------------------------------------------------+--------------+----------+----------+
|id       |tytul                                                 |cos_similarity|IOU       |prediction|
+---------+------------------------------------------------------+--------------+----------+----------+
|tt0449088|Pirates of the Caribbean: At World's End              |1.0           |1.0       |1.0       |
|tt0325980|Pirates of the Caribbean: The Curse of the Black Pearl|1.0           |1.0       |1.0       |
|tt0383574|Pirates of the Caribbean: Dead Man's Chest            |1.0           |0.8181818 |0.9091    |
|tt1298650|Pirates of the Caribbean: On Stranger Tides           |0.8           |0.42857143|0.6143    |
|tt1790809|Pirates of the Caribbean: Dead Men Tell No Tales      |0.8           |0.42857143|0.6143    |
|tt0121765|Star Wars: Episode II - Attack of the Clones          |1.0           |0.0       |0.5       |
|tt0411477|Hellboy II: The Golden Army                          